# Analyzing the Impact of *Epic Systems Corp. v. Lewis*

By [Ben Hancock](https://www.law.com/author/profile/Ben-Hancock/)

This is part of a data journalism project by the *National Law Journal* to assess the impact of *Epic Systems Corp. v. Lewis*, a May 2018 decision by the U.S. Supreme Court that centered on the enforceability of mandatory workplace arbitration agreements. In collaboration with [Casetext](https://casetext.com/), ee identified and categorized nearly 100 federal district and appellate decisions from 2018 that cited *Epic* in order to assess how the case is changing the law around arbitration. You can read more about our methodology in the [README file](https://github.com/bhancock-alm/epic-impact/blob/master/README.md).

You are looking at a [Jupyter Notebook](https://jupyter.org/), a saved interactive data analysis and computing session that uses the Python programming language. Our goal in presenting this to our audience is to "show our work" and give greater transparency into the approach we took in reporting an issue with wide public interest and significance. Even if you're not a programmer, our hope is that you can follow along -- with the help of plain-language comments interspersed below -- and see the steps we took to arrive at the figures cited in our [final article](https://www.law.com/nationallawjournal/2019/02/28/epic-impact-how-a-major-scotus-decision-in-favor-of-arbitration-is-shaping-the-landscape-for-workplace-lawsuits). At bottom, we show you how we generated one of the main data visualizations presented in the story. If you're just interested in seeing the underlying data, [that's here too](https://github.com/bhancock-alm/epic-impact/blob/master/epic_data_full.csv).

This is an experiment for our newsroom that aims to foster conversation with our audience in a new way. If you have questions, comments, or critiques, feel free to contact me at [bhancock@alm.com](mailto:bhancock@alm.com).

---

Import `pandas`, the [Python Data Analysis Library](https://pandas.pydata.org/), and several tools for data visualization

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Import the data we collected on decisions citing *Epic*

In [2]:
data = pd.read_csv('epic_data_full.csv')

Get a feel for the data set

In [3]:
data.shape

(92, 15)

In [4]:
data.columns

Index(['decision_id', 'case_name_short', 'case_name_long', 'jurisdiction_code',
       'jurisdiction_long', 'court_type', 'decide_date', 'casetext_url',
       'class_action', 'case_type', 'legal_issue', 'cite_context',
       'cite_excerpt', 'outcome_code', 'decision_favor'],
      dtype='object')

In [5]:
data.head()

,decision_id,case_name_short,case_name_long,jurisdiction_code,jurisdiction_long,court_type,decide_date,casetext_url,class_action,case_type,legal_issue,cite_context,cite_excerpt,outcome_code,decision_favor
0,1,"Goplin v. Weconnect, Inc.","BROOKS GOPLIN, Plaintiff-Appellee, v. WECONNEC...",7th Cir.,United States Court of Appeals For the Seventh...,circuit,2018-06-20,https://casetext.com/case/goplin-v-weconnect-i...,True,wage_hour,arbitration,in-passing,"Goplin also argued, and the district court agr...",no_arb,ptf
1,2,"Riederer v. United Healthcare Servs., Inc.","SANDRA RIEDERER, Plain tiff-Appellee, v. UNITE...",7th Cir.,United States Court of Appeals For the Seventh...,circuit,2018-07-23,https://casetext.com/case/riederer-v-united-he...,True,wage_hour,arbitration,supporting,"The decision of the district court is vacated,...",other,def
2,3,Hernandez v. Acosta Tractors Inc.,"JULIO HERNANDEZ HERNANDEZ, and all others simi...",11th Cir.,UNITED STATES COURT OF APPEALS FOR THE ELEVENT...,circuit,2018-08-07,https://casetext.com/case/hernandez-v-acosta-t...,False,wage_hour,arbitration,in-passing,"We find no basis in the FAA, the caselaw, or a...",other,def
3,4,"Gaffers v. Kelly Servs., Inc.","JONATHAN GAFFERS, Plaintiff-Appellee, v. KELLY...",6th Cir.,UNITED STATES COURT OF APPEALS FOR THE SIXTH C...,circuit,2018-08-14,https://casetext.com/case/gaffers-v-kelly-serv...,True,wage_hour,arbitration,supporting,The Supreme Court recently held that the Natio...,arb_compel,def
4,5,Dish Network L.L.C. v. Ray,"DISH NETWORK L.L.C. and ECHOSPHERE, L.L.C. Pla...",10th Cir.,UNITED STATES COURT OF APPEALS FOR THE TENTH C...,circuit,2018-08-20,https://casetext.com/case/dish-network-v-ray-1,True,wage_hour,arbitration,supporting,"By contrast, whether a claimant can proceed on...",other,ptf


How many circuit decisions, how many district?

In [6]:
data['court_type'].value_counts()

district    75
circuit     17
Name: court_type, dtype: int64

How many dealt with arbitration?

In [7]:
data.pivot_table(index=['court_type', 'legal_issue'],
                 values='decision_id',
                 aggfunc='count'
                )

decision_id
court_type legal_issue             
circuit    arbitration           11
           other                  6
district   arbitration           63
           other                 12

How many dealt with arbitration *and* workplace issues?  
Workplace issues include wage & hour, sexual harassment, worker classification, and discrimination

In [8]:
workplace = data.loc[(data['case_type'] == 'wage_hour') |
                     (data['case_type'] == 'sex_harass') |
                     (data['case_type'] == 'classification') |
                     (data['case_type'] == 'discrimination')]

In [9]:
workplace.pivot_table(index=['court_type', 'legal_issue'],
                      values='decision_id',
                      aggfunc='count'
                     )

decision_id
court_type legal_issue             
circuit    arbitration           10
district   arbitration           49
           other                  5

How many decisions went in favor of the defendant?

In [10]:
favor = pd.DataFrame({'case_count': data['decision_favor'].value_counts(),
                      'pct': data['decision_favor'].value_counts() / data['decision_favor'].value_counts().sum() * 100
                     })

favor.at['total', 'case_count'] = data['decision_favor'].value_counts().sum()
favor.at['total', 'pct'] = favor['pct'].sum()

favor

,case_count,pct
def,58.0,63.043478
ptf,24.0,26.086957
mixed,10.0,10.869565
total,92.0,100.000000
